In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
new_application_df = application_df.drop(columns=['EIN','NAME'])
new_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_types_to_replace = ['T17','T15','T29','T14','T25','T2','T12','T13','T9']

# Replace in dataframe
for app in application_types_to_replace:
    new_application_df['APPLICATION_TYPE'] = new_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
new_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
new_app_vc=new_application_df['CLASSIFICATION'].value_counts()
classifications_to_replace=new_app_vc[new_app_vc<1500].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    new_application_df['CLASSIFICATION'] = new_application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
new_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
application_cat = new_application_df.dtypes[new_application_df.dtypes == "object"].index.tolist()
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df  = pd.DataFrame(enc.fit_transform(new_application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
application_dummies = new_application_df.merge(encode_df,left_index=True, right_index=True)
application_dummies = application_dummies.drop(application_cat,1)

/var/folders/6p/r44x4tbj6wb4xd9n21b6_kqr0000gn/T/ipykernel_12558/337199410.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_dummies = application_dummies.drop(application_cat,1)


In [6]:
y = application_dummies["IS_SUCCESSFUL"].values
X = application_dummies.drop(["IS_SUCCESSFUL"],1).values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/var/folders/6p/r44x4tbj6wb4xd9n21b6_kqr0000gn/T/ipykernel_12558/4222660655.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = application_dummies.drop(["IS_SUCCESSFUL"],1).values


In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
number_input_features = len(X_train[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=100, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 7,751
Trainable params: 7,751
Non-trainable params: 0
_________________________________________________________________


2022-05-22 00:31:52.786085: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5400 - accuracy: 0.7386
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5395 - accuracy: 0.7385
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5392 - accuracy: 0.7388
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5396 - accuracy: 0.7394
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5391 - accuracy: 0.7392
Epoch 6/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5389 - accuracy: 0.7389
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5390 - accuracy: 0.7379
Epoch 8/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5385 - accuracy: 0.7393
Epoch 9/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5389 - accuracy: 0.7390
Epoch 10/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5388 - accuracy: 0.7399

In [13]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5539 - accuracy: 0.7245 - 413ms/epoch - 2ms/step
Loss: 0.5539400577545166, Accuracy: 0.7245481014251709


In [14]:
number_input_features = len(X_train[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=70, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 70)                3080      
                                                                 
 dense_5 (Dense)             (None, 60)                4260      
                                                                 
 dense_6 (Dense)             (None, 20)                1220      
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 8,581
Trainable params: 8,581
Non-trainable params: 0
_________________________________________________________________


In [15]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=75)

Epoch 1/75
804/804 [==============================] - 4s 3ms/step - loss: 0.5805 - accuracy: 0.7170
Epoch 2/75
804/804 [==============================] - 3s 3ms/step - loss: 0.5626 - accuracy: 0.7269
Epoch 3/75
804/804 [==============================] - 3s 3ms/step - loss: 0.5592 - accuracy: 0.7292
Epoch 4/75
804/804 [==============================] - 3s 4ms/step - loss: 0.5563 - accuracy: 0.7310
Epoch 5/75
804/804 [==============================] - 3s 4ms/step - loss: 0.5550 - accuracy: 0.7319
Epoch 6/75
804/804 [==============================] - 3s 4ms/step - loss: 0.5533 - accuracy: 0.7325
Epoch 7/75
804/804 [==============================] - 4s 4ms/step - loss: 0.5533 - accuracy: 0.7329
Epoch 8/75
804/804 [==============================] - 3s 4ms/step - loss: 0.5521 - accuracy: 0.7315
Epoch 9/75
804/804 [==============================] - 3s 4ms/step - loss: 0.5514 - accuracy: 0.7331
Epoch 10/75
804/804 [==============================] - 4s 4ms/step - loss: 0.5506 - accuracy: 0.7330

In [17]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5508 - accuracy: 0.7261 - 521ms/epoch - 2ms/step
Loss: 0.5508170127868652, Accuracy: 0.726064145565033


In [18]:
number_input_features = len(X_train[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=70, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="relu"))

nn.add(tf.keras.layers.Dense(units=40, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 70)                3080      
                                                                 
 dense_9 (Dense)             (None, 60)                4260      
                                                                 
 dense_10 (Dense)            (None, 40)                2440      
                                                                 
 dense_11 (Dense)            (None, 20)                820       
                                                                 
 dense_12 (Dense)            (None, 1)                 21        
                                                                 
Total params: 10,621
Trainable params: 10,621
Non-trainable params: 0
_________________________________________________________________


In [19]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5665 - accuracy: 0.7242
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5548 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5525 - accuracy: 0.7327
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5510 - accuracy: 0.7331
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5497 - accuracy: 0.7336
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5481 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5482 - accuracy: 0.7331
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5475 - accuracy: 0.7340
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5465 - accuracy: 0.7342
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5460 - accura

In [21]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5565 - accuracy: 0.7259 - 1s/epoch - 4ms/step
Loss: 0.5564785003662109, Accuracy: 0.7259474992752075


In [ ]:
nn.save("AlphabetSoupCharity_Organization.H5")